In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle
import warnings

warnings.filterwarnings("ignore")

# 1. Load and Prepare Data
def load_data():
    url = "https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv"
    data = pd.read_csv(url)

    X = data.drop('Outcome', axis=1)
    y = data['Outcome']

    return X, y

# 2. Train Model
def train_model():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = DecisionTreeClassifier(
        max_depth=4,
        min_samples_split=10,
        random_state=42
    )
    model.fit(X_train, y_train)

    # Save model and feature names
    pickle.dump(model, open('diabetes_model.pkl', 'wb'))
    pickle.dump(X.columns.tolist(), open('diabetes_features.pkl', 'wb'))

    return model, X.columns

# 3. Load or Train Model
try:
    model = pickle.load(open('diabetes_model.pkl', 'rb'))
    features = pickle.load(open('diabetes_features.pkl', 'rb'))
except:
    model, features = train_model()

# 4. Risk factor helper (FIXED CASE ISSUES)
def get_risk_factors(data):
    factors = []
    if data['Glucose'].values[0] > 140:
        factors.append("High glucose (>140 mg/dL)")
    if data['BMI'].values[0] > 30:
        factors.append("Obese (BMI >30)")
    if data['Age'].values[0] > 45:
        factors.append("Age >45 years")
    return ", ".join(factors) if factors else "No significant risk factors"

# 5. Prediction Function
def predict_diabetes(pregnancies, glucose, bp, skin_thickness, insulin, bmi, dpf, age):
    try:
        input_data = pd.DataFrame([[
            pregnancies, glucose, bp, skin_thickness, insulin, bmi, dpf, age
        ]], columns=features)

        proba = model.predict_proba(input_data)[0][1]
        prediction = 1 if proba >= 0.5 else 0

        result = {
            "Prediction": "Diabetic" if prediction else "Not Diabetic",
            "Probability": f"{proba:.1%}",
            "Risk Factors": get_risk_factors(input_data)
        }

        return result

    except Exception as e:
        return {"Error": f"Prediction failed: {str(e)}"}

# 6. Gradio Interface
inputs = [
    gr.Number(label="Pregnancies", minimum=0, maximum=20, value=1),
    gr.Number(label="Glucose (mg/dL)", minimum=50, maximum=300, value=100),
    gr.Number(label="Blood Pressure (mm Hg)", minimum=30, maximum=150, value=70),
    gr.Number(label="Skin Thickness (mm)", minimum=0, maximum=100, value=20),
    gr.Number(label="Insulin (μU/mL)", minimum=0, maximum=900, value=80),
    gr.Number(label="BMI", minimum=10, maximum=70, value=25),
    gr.Number(label="Diabetes Pedigree Function", minimum=0.08, maximum=2.5, step=0.01, value=0.5),
    gr.Number(label="Age", minimum=20, maximum=100, value=30)
]

outputs = gr.JSON(label="Prediction Results")

examples = [
    [1, 85, 66, 29, 0, 26.6, 0.351, 31],  # Not diabetic
    [6, 148, 72, 35, 0, 33.6, 0.627, 50]  # Diabetic
]

interface = gr.Interface(
    fn=predict_diabetes,
    inputs=inputs,
    outputs=outputs,
    examples=examples,
    title="Diabetes Prediction System",
    description="Predicts likelihood of diabetes using a decision tree model",
    allow_flagging="never"
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9183b6df8287ce93ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
